In [2]:
%load_ext autotime
import re

import numpy as np
import pandas as pd

import sqlalchemy as sal

time: 809 ms (started: 2021-09-29 09:30:06 +07:00)


In [3]:
engine = sal.create_engine(
    'sqlite:////Users/quoccuongnguyen/Documents/TDTU/YEAR-4/HK1-2021-2022/Massive-Data/Code/Database-SQLLite/518H0003-Ex4.db',
    echo=True)
conn = engine.connect()

time: 23.7 ms (started: 2021-09-29 09:30:08 +07:00)


In [4]:
# Q 1

metadata = sal.MetaData()
taixe = sal.Table('TAIXE', metadata,
                  sal.Column('MATX', sal.String, primary_key=True),
                  sal.Column('HOTEN', sal.String),
                  sal.Column('SDT', sal.String),
                  sal.Column('DIEMTB', sal.Float),
                  sal.Column('LXE', sal.String))

khachhang = sal.Table('KH', metadata,
                      sal.Column('MAKH', sal.String, primary_key=True),
                      sal.Column('HOTEN', sal.String),
                      sal.Column('SDT', sal.String))

datxe = sal.Table('DATXE', metadata,
                  sal.Column('MADX', sal.String, primary_key=True),
                  sal.Column('MAKH', None, sal.ForeignKey('KH.MAKH')),
                  sal.Column('DDI', sal.String),
                  sal.Column('DDEN', sal.String),
                  sal.Column('LXE', sal.String),
                  sal.Column('KC', sal.String),
                  sal.Column('GIA', sal.Integer),
                  sal.Column('TGDAT', sal.String))

doixe = sal.Table('DOIXE', metadata,
                  sal.Column('MADX', None, sal.ForeignKey('DATXE.MADX'), primary_key=True),
                  sal.Column('MATX', None, sal.ForeignKey('TAIXE.MATX'), primary_key=True),
                  sal.Column('TGBDAU', sal.String),
                  sal.Column('TGKTHUC', sal.String),
                  sal.Column('KETQUA', sal.String))

chuyendi = sal.Table('CHUYENDI', metadata,
                     sal.Column('MACD', sal.String, primary_key=True),
                     sal.Column('MADX', None, sal.ForeignKey('DOIXE.MADX')),
                     sal.Column('MATX', None, sal.ForeignKey('DOIXE.MATX')),
                     sal.Column('TGDI', sal.String),
                     sal.Column('TGDEN', sal.String),
                     sal.Column('GIA', sal.Integer),
                     sal.Column('HTTTOAN', sal.String),
                     sal.Column('DIEM', sal.Float))

metadata.create_all(engine)

2021-09-29 09:30:10,267 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-29 09:30:10,268 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("TAIXE")
2021-09-29 09:30:10,269 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-29 09:30:10,272 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("KH")
2021-09-29 09:30:10,273 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-29 09:30:10,274 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("DATXE")
2021-09-29 09:30:10,275 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-29 09:30:10,277 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("DOIXE")
2021-09-29 09:30:10,278 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-29 09:30:10,280 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CHUYENDI")
2021-09-29 09:30:10,282 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-29 09:30:10,284 INFO sqlalchemy.engine.Engine COMMIT
time: 28.6 ms (started: 2021-09-29 09:30:10 +07:00)


In [5]:
# Q2.1
s = sal.select(chuyendi).where(chuyendi.c.DIEM >= 4.5)
for row in conn.execute(s):
    print('MACD:', row['MACD'], 'DIEM:', row['DIEM'])

2021-09-29 09:30:12,399 INFO sqlalchemy.engine.Engine SELECT "CHUYENDI"."MACD", "CHUYENDI"."MADX", "CHUYENDI"."MATX", "CHUYENDI"."TGDI", "CHUYENDI"."TGDEN", "CHUYENDI"."GIA", "CHUYENDI"."HTTTOAN", "CHUYENDI"."DIEM" 
FROM "CHUYENDI" 
WHERE "CHUYENDI"."DIEM" >= ?
2021-09-29 09:30:12,400 INFO sqlalchemy.engine.Engine [generated in 0.00165s] (4.5,)
MACD: CD001 DIEM: 4.5
time: 12.3 ms (started: 2021-09-29 09:30:12 +07:00)


In [6]:
# Q2.2
s = sal.select(khachhang, datxe).where(sal.and_(khachhang.c.MAKH == datxe.c.MAKH, khachhang.c.HOTEN == 'Nguyen Van A'))
for row in conn.execute(s):
    print('MADX: ' + row['MADX'] + '- DDI: ' + row['DDI'] + ' - DDEN: ' + row['DDEN'] + ' - TGDAT: ' + row['TGDAT'])

2021-09-29 09:30:13,198 INFO sqlalchemy.engine.Engine SELECT "KH"."MAKH", "KH"."HOTEN", "KH"."SDT", "DATXE"."MADX", "DATXE"."MAKH" AS "MAKH_1", "DATXE"."DDI", "DATXE"."DDEN", "DATXE"."LXE", "DATXE"."KC", "DATXE"."GIA", "DATXE"."TGDAT" 
FROM "KH", "DATXE" 
WHERE "KH"."MAKH" = "DATXE"."MAKH" AND "KH"."HOTEN" = ?
2021-09-29 09:30:13,200 INFO sqlalchemy.engine.Engine [generated in 0.00178s] ('Nguyen Van A',)
MADX: DX001- DDI: HCM - DDEN: HCm - TGDAT: 10:00
time: 9.5 ms (started: 2021-09-29 09:30:13 +07:00)


In [7]:
# Q2.3
s = sal.select(taixe, doixe).where(sal.and_(doixe.c.KETQUA == 'HUY', (doixe.c.MATX == taixe.c.MATX) == 'TX005'))
for row in conn.execute(s):
    print('MADX:' + row['MADX'] + ' - MATX:' + row['MATX'] + ' - HOTEN:' + row['HOTEN'] + '- SDT:' + row['SDT']
          + ' - TGBDAU:' + row['TGBDAU'] + ' - TGKTHUC:' + row['TGKTHUC'] + ' - KQUA:' + row['KQUA'])

2021-09-29 09:30:13,873 INFO sqlalchemy.engine.Engine SELECT "TAIXE"."MATX", "TAIXE"."HOTEN", "TAIXE"."SDT", "TAIXE"."DIEMTB", "TAIXE"."LXE", "DOIXE"."MADX", "DOIXE"."MATX" AS "MATX_1", "DOIXE"."TGBDAU", "DOIXE"."TGKTHUC", "DOIXE"."KETQUA" 
FROM "TAIXE", "DOIXE" 
WHERE "DOIXE"."KETQUA" = ? AND ("DOIXE"."MATX" = "TAIXE"."MATX") = ?
2021-09-29 09:30:13,875 INFO sqlalchemy.engine.Engine [generated in 0.00185s] ('HUY', 'TX005')
time: 6.81 ms (started: 2021-09-29 09:30:13 +07:00)


In [ ]:
# Q2.4
s = sal.select(khachhang,datxe,doixe,chuyendi,sal.func.count(chuyendi.c.MACD).label('SLCD').groupby(khachhang.c.MAKH)\
               .order_by('MAKH',sal.asc('SLCD'))).where()
